# Sequential U-Turn Plot Generation

This notebook is a clean, single-purpose plotting workspace. It loads existing sequential U-turn trajectories and produces a "cartoon" horizontal figure showing the progression from the clean image through several U-turns to the final image.

Notes:
- All paths are hardcoded for the SSH cluster layout.
- This notebook does **not** generate new data; it only reads existing results.


In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

plt.style.use('seaborn-v0_8-whitegrid')
print('Setup complete.')


In [ ]:
# --- DISCOVERY: find available images / noise levels / trajectories ---
# This is lightweight (no image loading). You can run it before setting config.

DISCOVERY_BASE = '/work/pcsl/Noam/sequential_diffusion/results/sequential_uturns'
DISCOVERY_MAX_IMAGES = None      # e.g., 50 to limit
DISCOVERY_MAX_TRAJ = 3           # sample a few trajectories per noise level to estimate max U-turns

def _max_uturn_idx(traj_dir):
    jpeg_paths = glob.glob(os.path.join(traj_dir, 'uturn_*.jpeg'))
    png_paths = glob.glob(os.path.join(traj_dir, 'uturn_*.png'))
    paths = jpeg_paths + png_paths
    if not paths:
        return None
    idxs = []
    for p in paths:
        try:
            idx = int(os.path.splitext(os.path.basename(p))[0].split('_')[-1])
            idxs.append(idx)
        except Exception:
            pass
    return max(idxs) if idxs else None

def scan_results(base_dir, max_images=None, max_traj_per_noise=3):
    images = sorted([d for d in glob.glob(os.path.join(base_dir, '*')) if os.path.isdir(d)])
    if max_images is not None:
        images = images[:max_images]
    summary = {}
    for img_dir in images:
        img_name = os.path.basename(img_dir)
        noise_dirs = sorted(glob.glob(os.path.join(img_dir, 'noise_step_*')))
        if not noise_dirs:
            continue
        summary[img_name] = {}
        for nd in noise_dirs:
            noise_name = os.path.basename(nd)
            try:
                noise_step = int(noise_name.split('_')[-1])
            except Exception:
                noise_step = noise_name
            traj_dirs = sorted(glob.glob(os.path.join(nd, 'trajectory_*')))
            num_traj = len(traj_dirs)
            sampled = traj_dirs[:max_traj_per_noise] if max_traj_per_noise else traj_dirs
            max_uturns = []
            for td in sampled:
                m = _max_uturn_idx(td)
                if m is not None:
                    max_uturns.append(m)
            max_uturn = max(max_uturns) if max_uturns else None
            summary[img_name][noise_step] = {
                'num_trajectories': num_traj,
                'sampled_for_max_uturn': len(sampled),
                'max_uturn_observed': max_uturn,
            }
    return summary

def print_summary(summary):
    for img_name in sorted(summary.keys()):
        noise_steps = sorted(summary[img_name].keys())
        print('\nImage: {}'.format(img_name))
        print('  Noise steps: {}'.format(noise_steps))
        for ns in noise_steps:
            info = summary[img_name][ns]
            print('    noise_step_{}: trajectories={}, max_uturn~{} (sampled {})'.format(
                ns, info['num_trajectories'], info['max_uturn_observed'], info['sampled_for_max_uturn']))

summary = scan_results(DISCOVERY_BASE, max_images=DISCOVERY_MAX_IMAGES, max_traj_per_noise=DISCOVERY_MAX_TRAJ)
print_summary(summary)


In [ ]:
# --- CONFIGURATION ---
BASE_RESULTS_DIR = '/work/pcsl/Noam/sequential_diffusion/results/sequential_uturns'
IMAGE_NAME = 'ILSVRC2012_val_00000729'
NOISE_STEP = 200

# If AUTO_SELECT is True, the notebook will pick a trajectory automatically
# using a simple drift/smoothness proxy from CLIP patch embeddings.
AUTO_SELECT = False
TRAJECTORY_INDEX = 0

# Steps to show in the cartoon (use -1 for the final step).
STEPS_TO_PLOT = [0, 1, 2, 3, 5, 10, -1]

# If -1 is not provided and steps exceed the length, we fall back to this jump size.
JUMP_FALLBACK = 5

# Output filename (saved inside the chosen trajectory directory).
OUTPUT_FIG_NAME = 'cartoon_uturns_horizontal.png'


In [ ]:
def list_trajectory_dirs(base_dir, image_name, noise_step):
    image_dir = os.path.join(base_dir, image_name, f'noise_step_{noise_step}')
    pattern = os.path.join(image_dir, 'trajectory_*')
    return sorted(glob.glob(pattern))

def load_trajectory_images(trajectory_dir):
    jpeg_paths = glob.glob(os.path.join(trajectory_dir, 'uturn_*.jpeg'))
    png_paths = glob.glob(os.path.join(trajectory_dir, 'uturn_*.png'))
    paths = sorted(jpeg_paths + png_paths, key=lambda p: int(os.path.splitext(os.path.basename(p))[0].split('_')[-1]))
    images = [Image.open(p).convert('RGB') for p in paths]
    return images, paths

def load_embeddings(trajectory_dir):
    npz_path = os.path.join(trajectory_dir, 'trajectory_data.npz')
    if not os.path.exists(npz_path):
        return None
    data = np.load(npz_path)
    return data['embeddings']  # shape [steps+1, 49, 768]

def compute_patch_drift(embeddings, step_a=0, step_b=-1):
    if step_b < 0:
        step_b = embeddings.shape[0] - 1
    diff = embeddings[step_b] - embeddings[step_a]
    return np.linalg.norm(diff, axis=-1).mean()

def compute_roughness(embeddings):
    diffs = embeddings[1:] - embeddings[:-1]
    return np.linalg.norm(diffs, axis=-1).mean()

def score_trajectory(embeddings):
    drift = compute_patch_drift(embeddings, 0, -1)
    rough = compute_roughness(embeddings)
    score = drift / (rough + 1e-8)
    return drift, rough, score

def select_best_trajectory(traj_dirs):
    best = None
    for d in traj_dirs:
        emb = load_embeddings(d)
        if emb is None:
            continue
        drift, rough, score = score_trajectory(emb)
        if best is None or score > best['score']:
            best = dict(dir=d, drift=drift, rough=rough, score=score)
    return best

def resolve_steps(num_steps, steps_to_plot, jump_fallback=5):
    steps = []
    for s in steps_to_plot:
        if s == -1:
            steps.append(num_steps - 1)
        else:
            steps.append(s)
    steps = [s for s in steps if 0 <= s < num_steps]
    if not steps:
        steps = list(range(0, num_steps, jump_fallback))
    seen = set()
    ordered = []
    for s in steps:
        if s not in seen:
            seen.add(s)
            ordered.append(s)
    return ordered

def plot_horizontal(images, steps, title, out_path):
    n = len(steps)
    fig, axes = plt.subplots(1, n, figsize=(2.6 * n, 2.6))
    if n == 1:
        axes = [axes]
    for ax, step in zip(axes, steps):
        ax.imshow(images[step])
        ax.set_title(f'step {step}', fontsize=10)
        ax.axis('off')
    fig.suptitle(title, fontsize=14)
    fig.tight_layout()
    fig.savefig(out_path, dpi=200, bbox_inches='tight')
    print(f'✅ Saved figure to: {out_path}')


In [ ]:
# --- MAIN ---
traj_dirs = list_trajectory_dirs(BASE_RESULTS_DIR, IMAGE_NAME, NOISE_STEP)
if not traj_dirs:
    raise FileNotFoundError('No trajectories found for this image/noise setting.')

if AUTO_SELECT:
    best = select_best_trajectory(traj_dirs)
    if best is None:
        raise FileNotFoundError('No trajectories with embeddings found.')
    trajectory_dir = best['dir']
    print('Selected trajectory: {}'.format(trajectory_dir))
    print('Drift={:.3f}, Roughness={:.3f}, Score={:.3f}'.format(best['drift'], best['rough'], best['score']))
else:
    trajectory_dir = traj_dirs[TRAJECTORY_INDEX]
    print('Using trajectory: {}'.format(trajectory_dir))

images, paths = load_trajectory_images(trajectory_dir)
if not images:
    raise FileNotFoundError('No uturn_*.jpeg/png images found in trajectory.')

steps = resolve_steps(len(images), STEPS_TO_PLOT, JUMP_FALLBACK)
title = 'Sequential U-Turns | {} | t={} | {}'.format(IMAGE_NAME, NOISE_STEP, os.path.basename(trajectory_dir))
out_path = os.path.join(trajectory_dir, OUTPUT_FIG_NAME)
plot_horizontal(images, steps, title, out_path)
